In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
import sklearn.metrics as skm
from tqdm.notebook import tqdm
import sklearn.feature_extraction.text
from transformers import AutoModelForMaskedLM, AutoConfig, BertTokenizer, LineByLineTextDataset

from shopee_utils import *
from train_utils import *
import codecs
from torch.utils.data.dataset import Dataset

In [3]:
BERT_CONFIG_PATH = './indobert-base-p2'
PATH = Path('/home/slex/data/shopee')

In [4]:
train_df = pd.read_csv(PATH/'train.csv')

In [5]:
model = AutoModelForMaskedLM.from_pretrained(BERT_CONFIG_PATH)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at ./indobert-base-p2 and are newly initialized: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokenizer = BertTokenizer.from_pretrained(BERT_CONFIG_PATH)

In [7]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, lines, block_size):
        batch_encoding = tokenizer(lines, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]
        self.examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in self.examples]
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [8]:
dataset=TextDataset(tokenizer, train_df.title.to_list(), 128)

In [9]:
tokenizer.decode(dataset[0]['input_ids'])

'[CLS] paper bag victoria secret [SEP]'

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./LanguageModel",
    overwrite_output_dir=True,
    num_train_epochs=6,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


In [12]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
 16%|█▌        | 501/3216 [01:19<05:36,  8.08it/s]

{'loss': 2.1224, 'learning_rate': 4.222636815920398e-05, 'epoch': 0.93}


 31%|███       | 1001/3216 [02:38<05:41,  6.48it/s]

{'loss': 1.5216, 'learning_rate': 3.445273631840796e-05, 'epoch': 1.87}


 47%|████▋     | 1501/3216 [03:57<04:34,  6.25it/s]

...

 93%|█████████▎| 3001/3216 [07:51<00:36,  5.88it/s]

{'loss': 1.1616, 'learning_rate': 3.358208955223881e-06, 'epoch': 5.6}


100%|██████████| 3216/3216 [08:26<00:00,  6.35it/s]

{'train_runtime': 506.4408, 'train_samples_per_second': 6.35, 'epoch': 6.0}


TrainOutput(global_step=3216, training_loss=1.4209974250983244, metrics={'train_runtime': 506.4408, 'train_samples_per_second': 6.35, 'epoch': 6.0})

In [13]:
trainer.save_model('./finetuned_indobert-base-p2')